In [1]:
import stable_whisper
import pickle
from tqdm import tqdm

In [2]:
class CFG:
    pth="./wav/Tamako.wav"
    save_folder='slices'
    list_file='list.txt'
    pkl_file='data.pkl'
    tmp_folder='./tmp'
    min_len=0.5
    max_len=10.5
    period=60
    RATE=16000
    language='Japanese'

In [3]:
import torchaudio
def Cut_Slices(result,p,f_list,filename=CFG.pth):
    global tot
    waveform,sample_rate = torchaudio.load(filename)
    data_=waveform[0]
    tot_size=len(data_)
    for item in result:
        if(item['no_speech_prob']>=0.5):
            continue
        data=data_[int((item['start'])*sample_rate):min(int((item['end'])*sample_rate),tot_size)]
        if(len(data)<int(CFG.min_len*sample_rate) or len(data)>int(CFG.max_len*sample_rate)):
            continue
        tot+=1
        torchaudio.save(CFG.save_folder+'/{}.wav'.format(tot),data.view(1,-1),sample_rate)
        # f_list.write('id={},peroid={}, {} to {}:'.format(tot,p,round(item['start'],1),round(item['end'],1))+item['text']+'\n')
        f_list.write('{}:'.format(tot)+item['text']+'\n')
        # if(tot==1191):
        #     print(item)
        
        

In [4]:
ptr=0
_,sample_rate = torchaudio.load(CFG.pth)
_=_[0]
tot_size=len(_)
psum=int(tot_size/(CFG.period*sample_rate)+1)

In [6]:
model = stable_whisper.load_model("medium")
for i in tqdm(range(int(tot_size/(CFG.period*sample_rate)+1))):
    if(ptr>=tot_size):
        break
    data=_[ptr:min(ptr+CFG.period*sample_rate,tot_size)]
    ptr+=CFG.period*sample_rate
    save_pth=CFG.tmp_folder+'/{}.wav'.format(i+1)
    torchaudio.save(save_pth.format(psum),data.view(1,-1),sample_rate)
    result = model.transcribe(save_pth,language=CFG.language,print_unstab=False)
    f = open(CFG.tmp_folder+'/{}.pkl'.format(i+1),'wb')
    data= pickle.dump(result,f,-1)
    f.close()
    

100%|██████████| 84/84 [31:11<00:00, 22.28s/it]


In [5]:
f_list=open(CFG.list_file,'w',encoding='utf-8')
tot=0
for i in tqdm(range(psum)):
    pkl_pth=CFG.tmp_folder+'/{}.pkl'.format(i+1)
    wav_pth=CFG.tmp_folder+'/{}.wav'.format(i+1)
    f = open(pkl_pth,'rb')
    result=pickle.load(f)
    result=result['segments']
    f.close()
    Cut_Slices(result,i+1,f_list,wav_pth)
f_list.close()

100%|██████████| 84/84 [00:05<00:00, 15.62it/s]
